<a href="https://colab.research.google.com/github/AnastasiaBrinati/Progetto-ML-23-24/blob/main/task3_federato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 3

#### imports

In [28]:
!pip install -q flwr["simulation"] tensorflow
!pip install -q flwr_datasets["vision"]

In [29]:
!pip install matplotlib

In [30]:
from typing import Dict, List, Tuple

import tensorflow as tf

import flwr as fl
from flwr.common import Metrics
from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth

from datasets import Dataset
from flwr_datasets import FederatedDataset

VERBOSE = 0
NUM_CLIENTS = 100

In [31]:
from sklearn.metrics import confusion_matrix

import pandas as pd

In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
!pip install -q -U keras-tuner
import keras_tuner as kt

#### model

Let's start by defining the model we want to federated. Since we will be working with MNIST, using a fully connected model is sufficient. You can of course customize this model.

In [33]:
# Build the actual model
def get_model():

  model = keras.Sequential()

  # (conv2 + pooling)*2
  model.add(layers.Conv2D(input_shape=(28,28,1), filters=1,kernel_size=5,activation='relu',padding='same'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(filters=48,kernel_size=5,activation='relu',padding='same'))

  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  # flatten layer
  model.add(layers.Flatten())

  # dense layers
  model.add(layers.Dense(units=64, activation="relu"))
  model.add(layers.Dense(units=80, activation="relu"))
  model.add(layers.Dense(units=32, activation="relu"))
  # output layer
  model.add(layers.Dense(10, activation='softmax'))

  # If your Yi's are integers, use sparse_categorical_crossentropy. Examples for above 3-class classification problem: [1] , [2], [3]
  # sparse_top_k_categorical_accuracy deafult K=5

  model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy", "sparse_categorical_accuracy", "sparse_top_k_categorical_accuracy"])
  return model

top_k_categorical_accuracy: This metric computes the number of times where the correct label is among the top k labels predicted (ranked by predicted scores);
In cases where two or more labels are assigned equal predicted scores, the labels with the highest indices will be chosen first. This might impact the result if the correct label falls after the threshold because of that.


#### build the model

In [34]:
model = get_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 1)         26        
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 14, 14, 1)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 48)        1248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 7, 7, 48)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2352)              0         
                                                                 
 dense_8 (Dense)             (None, 64)               

#### CLIENT

In [35]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, trainset, valset) -> None:
        # Create model
        self.model = get_model()
        self.trainset = trainset
        self.valset = valset

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.trainset, epochs=1, verbose=VERBOSE)
        return self.model.get_weights(), len(self.trainset), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        results = self.model.evaluate(self.valset, verbose=VERBOSE)

        loss = results[0]
        acc = results[1]
        cat_acc = results[2]
        topk_acc = results[3]

        return loss, len(self.valset), {"loss": loss, "accuracy": acc, "cat_acc": cat_acc, "topk_acc" : topk_acc}

In [36]:
def get_client_fn(dataset: FederatedDataset):
    """Return a function to construct a client.

    The VirtualClientEngine will execute this function whenever a client is sampled by
    the strategy to participate.
    """

    def client_fn(cid: str) -> fl.client.Client:
        """Construct a FlowerClient with its own dataset partition."""

        # Extract partition for client with id = cid
        client_dataset = dataset.load_partition(int(cid), "train")

        # Now let's split it into train (90%) and validation (10%)
        client_dataset_splits = client_dataset.train_test_split(test_size=0.1)

        trainset = client_dataset_splits["train"].to_tf_dataset(
            columns="image", label_cols="label", batch_size=32
        )
        valset = client_dataset_splits["test"].to_tf_dataset(
            columns="image", label_cols="label", batch_size=64
        )

        # Create and return client
        return FlowerClient(trainset, valset).to_client()

    return client_fn

In [37]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """Aggregation function for (federated) evaluation metrics, i.e. those returned by
    the client's evaluate() method."""
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    cat_accuracies = [num_examples * m["cat_acc"] for num_examples, m in metrics]
    topk_accuracies = [num_examples * m["topk_acc"] for num_examples, m in metrics]

    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples), "cat_acc": sum(cat_accuracies) / sum(examples), "topk_acc": sum(topk_accuracies) / sum(examples)}

In [38]:
def get_evaluate_fn(testset: Dataset):
    """Return an evaluation function for server-side (i.e. centralised) evaluation."""

    # The `evaluate` function will be called after every round by the strategy
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ):
        model = get_model()  # Construct the model
        model.set_weights(parameters)  # Update model with the latest parameters
        results = model.evaluate(testset, verbose=VERBOSE)

        loss = results[0]
        acc = results[1]
        cat_acc = results[2]
        topk_acc = results[3]

        return loss, {"accuracy": acc, "cat_acc": cat_acc, "topk_acc": topk_acc}

    return evaluate

#### settings and strategies

In [70]:
# Simulation parameters
settings = {
    "VERBOSE": 0,
    "NUM_CLIENTS": 5,
    "FRACTION_FIT": 0.8,
    "FRACTION_EVALUATE": 0.2,
    "MIN_FIT_CLIENTS": 5,
    "MIN_EVALUATE_CLIENTS": 2,
    "MIN_AVAILABLE_CLIENTS_FRACTION": 0.5,
    "STRATEGY_TYPE": 'QFedAvg', # QFedAvg, FedAvg, FedProx
    "NUM_ROUNDS": 10
}

In [79]:
# Create FedAvg strategy

def get_strategy(centralized_testset):

  strategy1 = fl.server.strategy.FedAvg(
      fraction_fit=0.1,  # Sample 10% of available clients for training
      fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation
      min_fit_clients=10,  # Never sample less than 10 clients for training
      min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
      min_available_clients=int(
          NUM_CLIENTS * 0.75
      ),  # Wait until at least 75 clients are available
      evaluate_metrics_aggregation_fn=weighted_average,  # aggregates federated metrics
      evaluate_fn=get_evaluate_fn(centralized_testset),  # global evaluation function
  )

  # Create FedAvg strategy
  strategy2 = fl.server.strategy.FedProx(
      proximal_mu=1.0,
      fraction_fit=0.1,  # Sample 10% of available clients for training
      fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation
      min_fit_clients=10,  # Never sample less than 10 clients for training
      min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
      min_available_clients=int(
          NUM_CLIENTS * 0.75
      ),  # Wait until at least 75 clients are available
      evaluate_metrics_aggregation_fn=weighted_average,  # aggregates federated metrics
      evaluate_fn=get_evaluate_fn(centralized_testset),  # global evaluation function
  )

  # Create FedAvg strategy
  strategy3 = fl.server.strategy.QFedAvg(
      fraction_fit=0.1,  # Sample 10% of available clients for training
      fraction_evaluate=0.05,  # Sample 5% of available clients for evaluation
      min_fit_clients=10,  # Never sample less than 10 clients for training
      min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
      min_available_clients=int(
          NUM_CLIENTS * 0.75
      ),  # Wait until at least 75 clients are available
      evaluate_metrics_aggregation_fn=weighted_average,  # aggregates federated metrics
      evaluate_fn=get_evaluate_fn(centralized_testset),  # global evaluation function
  )

  return strategy3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
# Enable GPU growth in your main process
enable_tf_gpu_growth()

# Download MNIST dataset and partition it
mnist_fds = FederatedDataset(dataset="mnist", partitioners={"train": NUM_CLIENTS})
# Get the whole test set for centralised evaluation
centralized_testset = mnist_fds.load_full("test").to_tf_dataset(
    columns="image", label_cols="label", batch_size=64
)

strategy = get_strategy(centralized_testset)

#### TESTING

In [17]:
c_data = mnist_fds.load_partition(0, "train")

client_dataset_splits = c_data.train_test_split(test_size=0.1)

x_tr = client_dataset_splits["train"].to_tf_dataset(columns="image", label_cols="label", batch_size=32)
x_ts = client_dataset_splits["test"].to_tf_dataset(columns="image", label_cols="label", batch_size=64)

In [18]:
m = get_model()
h = m.fit(x_tr)

17/17 [==============================] - 2s 27ms/step - loss: 3.7163 - accuracy: 0.2630 - sparse_categorical_accuracy: 0.2630 - sparse_top_k_categorical_accuracy: 0.7037


In [19]:
h.history

{'loss': [3.71625018119812],
 'accuracy': [0.2629629671573639],
 'sparse_categorical_accuracy': [0.2629629671573639],
 'sparse_top_k_categorical_accuracy': [0.7037037014961243]}

#### SIMULATION

In [ ]:
# With a dictionary, you tell Flower's VirtualClientEngine that each
# client needs exclusive access to these many resources in order to run
client_resources = {"num_cpus": 2, "num_gpus": 0.0}

# Start simulation
history = fl.simulation.start_simulation(
    client_fn=get_client_fn(mnist_fds),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
    actor_kwargs={
        "on_actor_init_fn": enable_tf_gpu_growth  # Enable GPU growth upon actor init.
    },
)

#### PLOT

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(history):
  fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(19, 5))

  ax1.plot(pd.DataFrame(history.c_loss), label='loss_centralizzata')
  ax1.plot(pd.DataFrame(history.loss_avg), label='loss_avg')
  ax1.plot(pd.DataFrame(history.loss_prox), label='loss_prox')
  ax1.plot(pd.DataFrame(history.loss_qavg), label='loss_qavg')

  ax2.plot(pd.DataFrame(history.c_mse), label='mse_centralized')
  ax2.plot(pd.DataFrame(history.mse_avg), label='mse_avg')
  ax2.plot(pd.DataFrame(history.mse_prox), label='mse_prox')
  ax2.plot(pd.DataFrame(history.mse_qavg), label='mse_qavg')

  ax3.plot(pd.DataFrame(history.c_mape), label='mape_centralized')
  ax3.plot(pd.DataFrame(history.mape_avg), label='mape_avg')
  ax3.plot(pd.DataFrame(history.mape_prox), label='mape_prox')
  ax3.plot(pd.DataFrame(history.mape_qavg), label='mape_qavg')

  ax4.plot(pd.DataFrame(history.c_msle), label='msle_centralized')
  ax4.plot(pd.DataFrame(history.msle_avg), label='msle_avg')
  ax4.plot(pd.DataFrame(history.msle_prox), label='msle_prox')
  ax4.plot(pd.DataFrame(history.msle_qavg), label='msle_qavg')

  ax5.plot(pd.DataFrame(history.c_r2), label='r2_centralized')
  ax5.plot(pd.DataFrame(history.r2_avg), label='r2_avg')
  ax5.plot(pd.DataFrame(history.r2_prox), label='r2_prox')
  ax5.plot(pd.DataFrame(history.r2_qavg), label='r2_qavg')

  ax1.set(xlabel='Epoch', ylabel='loss')
  ax1.legend(loc="upper right")
  ax1.grid(True)

  ax2.set(xlabel='Epoch', ylabel='mse')
  ax2.legend(loc="upper right")
  ax2.grid(True)

  ax3.set(xlabel='Epoch', ylabel='mape')
  ax3.legend(loc="upper right")
  ax3.grid(True)

  ax4.set(xlabel='Epoch', ylabel='mean_squared_logarithmic_error')
  ax4.legend(loc="upper right")
  ax4.grid(True)

  ax5.set(xlabel='Epoch', ylabel='r2_score')
  ax5.legend(loc="upper right")
  ax5.grid(True)

  fig.tight_layout()

## 100 clients

#### AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_avg'})
h_fedavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_avg'})
h_avg = h_fedavg.drop(0)
h_avg

#### PROX

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_prox'})
h_prox = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_prox'})
h_prox = h_prox.drop(0)
h_prox

#### QAVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_qavg'})
h_fedQavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_qavg'})
h_qavg = h_fedQavg.drop(0)
h_qavg

## 10 clients

#### AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_avg'})
h_fedavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_avg'})
h_avg = h_fedavg.drop(0)
h_avg

#### PROX

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_prox'})
h_prox = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_prox'})
h_prox = h_prox.drop(0)
h_prox

#### QAVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_qavg'})
h_fedQavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_qavg'})
h_qavg = h_fedQavg.drop(0)
h_qavg

## 5 clients

#### AVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_avg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_avg'})
h_fedavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_avg'})
h_avg = h_fedavg.drop(0)
h_avg

#### PROX

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_prox'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_prox'})
h_prox = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_prox'})
h_prox = h_prox.drop(0)
h_prox

#### QAVG

In [ ]:
h = pd.DataFrame(history.metrics_centralized['accuracy']).drop([0], axis=1).rename(columns={1: 'accuracy_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['cat_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'categorical_accuracy_qavg'})
h = pd.concat([h, pd.DataFrame(history.metrics_centralized['topk_acc']).drop([0], axis=1)], axis=1).rename(columns={1: 'topk_categorical_accuracy_qavg'})
h_fedQavg = pd.concat([h, pd.DataFrame(history.losses_centralized).drop([0], axis=1)], axis=1).rename(columns={1: 'loss_qavg'})
h_qavg = h_fedQavg.drop(0)
h_qavg

# DATA

#### 100

In [ ]:
#AVG 100 clients

data_avg_100 = [ [0,0,0,0],
    [0.1308,	0.1308,	0.5753,	2.505817],
    [0.3246,	0.3246,	0.7525,	2.000379],
    [0.4898,	0.4898,	0.9141,	1.478582],
    [0.6543,	0.6543,	0.9668,	1.046845],
    [0.7552,	0.7552,	0.9785,	0.750211],
    [0.8001,	0.8001,	0.9833,	0.623913],
    [0.8352,	0.8352,	0.9854,	0.529405],
    [0.8536,	0.8536,	0.9881,	0.464159],
    [0.8750,	0.8750,	0.9906,	0.413245],
    [0.8866,	0.8866,	0.9899,	0.379287]]

h100_avg = pd.DataFrame(data_avg_100, columns=['accuracy_avg', 'cat_accuracy_avg', 'topk_accuracy_avg','loss_avg']).drop(0)
h100_avg

In [ ]:
#PROX 100 clients

data_prox_100 = [ [0,0,0,0],
    [0.2901,	0.2901,	0.7522,	2.081158],
    [0.6465,	0.6465,	0.9611,	1.047874],
    [0.7844,	0.7844,	0.9783,	0.701940],
    [0.8335,	0.8335,	0.9835,	0.556559],
    [0.8619,	0.8619,	0.9880,	0.466229],
    [0.8735,	0.8735,	0.9896,	0.419731],
    [0.8920,	0.8920,	0.9907,	0.368518],
    [0.8982,	0.8982,	0.9920,	0.342408],
    [0.9058,	0.9058,	0.9936,	0.307516],
    [0.9137,	0.9137,	0.9942,	0.284420]]

h100_prox = pd.DataFrame(data_prox_100, columns=['accuracy_prox', 'cat_accuracy_prox', 'topk_accuracy_prox','loss_prox']).drop(0)
h100_prox

In [ ]:
#QAVG 100 clients

data_qavg_100 = [[0,0,0,0],
    [0.1736,	0.1736,	0.6223,	3.052144],
    [0.2191,	0.2191,	0.6136,	2.138689],
    [0.3004,	0.3004,	0.7422,	2.040695],
    [0.4299,	0.4299,	0.8810,	1.813220],
    [0.5358,	0.5358,	0.9144,	1.393641],
    [0.5787,	0.5787,	0.9514,	1.139368],
    [0.6509,	0.6509,	0.9717,	0.912194],
    [0.7234,	0.7234,	0.9787,	0.800335],
    [0.7767,	0.7767,	0.9836,	0.690224],
    [0.8106,	0.8106,	0.9866,	0.611684]]

h100_qavg = pd.DataFrame(data_qavg_100, columns=['accuracy_qavg', 'cat_accuracy_qavg', 'topk_accuracy_qavg','loss_qavg']).drop(0)
h100_qavg

#### 10

In [ ]:
# AVG 10 clients

data_avg_10 = [[0,0,0,0],
    [0.1396,	0.1396,	0.6178,	3.834348],
    [0.3144,	0.3144,	0.8005,	1.981152],
    [0.4015,	0.4015,	0.8854,	1.618643],
    [0.6595,	0.6595,	0.9510,	1.211388],
    [0.7933,	0.7933,	0.9725,	0.757223],
    [0.8624,	0.8624,	0.9874,	0.493698],
    [0.8788,	0.8788,	0.9893,	0.412210],
    [0.9020,	0.9020,	0.9929,	0.320931],
    [0.9135,	0.9135,	0.9940,	0.279641],
    [0.9238,	0.9238,	0.9942,	0.248362]]

h10_avg = pd.DataFrame(data_avg_10, columns=['accuracy_avg', 'cat_accuracy_avg', 'topk_accuracy_avg','loss_avg']).drop(0)
h10_avg

In [ ]:
# PROX 10 clients

data_prox_10 = [[ 0,0,0,0],
    [0.2111,	0.2111,	0.7899,	2.065002],
    [0.5874,	0.5874,	0.9376,	1.274748],
    [0.7182,	0.7182,	0.9709,	0.862452],
    [0.7933,	0.7933,	0.9845,	0.654375],
    [0.8403,	0.8403,	0.9894,	0.518748],
    [0.8488,	0.8488,	0.9907,	0.462882],
    [0.8767,	0.8767,	0.9936,	0.398931],
    [0.8907,	0.8907,	0.9940,	0.352000],
    [0.9002,	0.9002,	0.9956,	0.322833],
    [0.9140,	0.9140,	0.9964,	0.288012 ]]

h10_prox = pd.DataFrame(data_prox_10, columns=['accuracy_prox', 'cat_accuracy_prox', 'topk_accuracy_prox','loss_prox']).drop(0)
h10_prox

In [83]:
# QAVG 10 clients

data_qavg_10 = [[0,0,0,0],
    [0.1633,	0.1633,	0.6567,	2.810671],
    [0.3720,	0.3720,	0.8066,	1.824364],
    [0.4776,	0.4776,	0.9003,	1.510225],
    [0.6038,	0.6038,	0.9434,	1.189084],
    [0.6566,	0.6566,	0.9627,	1.028795],
    [0.7180,	0.7180,	0.9729,	0.871393],
    [0.7522,	0.7522,	0.9791,	0.775111],
    [0.7768,	0.7768,	0.9840,	0.687708],
    [0.7934,	0.7934,	0.9857,	0.634439],
    [0.8206,	0.8206,	0.9879,	0.569984]]

h10_qavg = pd.DataFrame(data_qavg_10, columns=['accuracy_qavg', 'cat_accuracy_qavg', 'topk_accuracy_qavg','loss_qavg']).drop(0)
h10_qavg

,accuracy_qavg,cat_accuracy_qavg,topk_accuracy_qavg,loss_qavg
1,0.1633,0.1633,0.6567,2.810671
2,0.3720,0.3720,0.8066,1.824364
3,0.4776,0.4776,0.9003,1.510225
4,0.6038,0.6038,0.9434,1.189084
5,0.6566,0.6566,0.9627,1.028795
6,0.7180,0.7180,0.9729,0.871393
7,0.7522,0.7522,0.9791,0.775111
8,0.7768,0.7768,0.9840,0.687708
9,0.7934,0.7934,0.9857,0.634439
10,0.8206,0.8206,0.9879,0.569984


#### 5

In [ ]:
# AVG 5 clients

data_avg_5 = [[0,0,0,0],
      [0.1791,	0.1791,	0.6482,	3.018399],
      [0.5450,	0.5450,	0.9487,	1.334101],
      [0.6940,	0.6940,	0.9768,	0.877807],
      [0.8395,	0.8395,	0.9855,	0.549155],
      [0.8814,	0.8814,	0.9902,	0.418776],
      [0.9069,	0.9069,	0.9922,	0.330590],
      [0.9189,	0.9189,	0.9935,	0.281983],
      [0.9254,	0.9254,	0.9939,	0.263710],
      [0.9348,	0.9348,	0.9954,	0.225585],
      [0.9402,	0.9402,	0.9962,	0.209047]]

h5_avg = pd.DataFrame(data_avg_5, columns=['accuracy_avg', 'cat_accuracy_avg', 'topk_accuracy_avg','loss_avg']).drop(0)
h5_avg

In [ ]:
# PROX 5 clients

data_prox_5 = [[0,0,0,0],
	[0.2039,	0.2039,	0.6811,	2.506892],
  [0.4199,	0.4199,	0.8906,	1.593910],
  [0.6056,	0.6056,	0.9365,	1.178540],
  [0.7671,	0.7671,	0.9683,	0.764252],
  [0.8279,	0.8279,	0.9796,	0.567366],
  [0.8617,	0.8617,	0.9851,	0.453980],
  [0.8819,	0.8819,	0.9860,	0.385182],
  [0.8988,	0.8988,	0.9905,	0.337637],
  [0.9077,	0.9077,	0.9913,	0.311508],
  [0.9183,	0.9183,	0.9929,	0.275550]]

h5_prox = pd.DataFrame(data_prox_5, columns=['accuracy_prox', 'cat_accuracy_prox', 'topk_accuracy_prox','loss_prox']).drop(0)
h5_prox

In [ ]:
# QAVG 5 clients

data_qavg_5 = [[0,0,0,0],
    [0.2820,	0.2820,	0.7107,	2.126181],
    [0.4595,	0.4595,	0.8653,	1.846541],
    [0.5911,	0.5911,	0.9333,	1.529425],
    [0.6826,	0.6826,	0.9648,	1.230552],
    [0.7282,	0.7282,	0.9770,	1.003095],
    [0.7655,	0.7655,	0.9802,	0.861222],
    [0.7898,	0.7898,	0.9832,	0.766520],
    [0.8096,	0.8096,	0.9848,	0.691888],
    [0.8200,	0.8200,	0.9860,	0.645418],
    [0.8285,	0.8285,	0.9868,	0.601402]]

h5_qavg = pd.DataFrame(data_qavg_5, columns=['accuracy_qavg', 'cat_accuracy_qavg', 'topk_accuracy_qavg','loss_qavg']).drop(0)
h5_qavg